# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sent_1880.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

217223

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

communale


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

453

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [9]:
%time bigram_phrases[sentences[78]]

CPU times: user 49 µs, sys: 8 µs, total: 57 µs
Wall time: 68.9 µs


[]

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [11]:
%time bigram_phraser[sentences[78]]

CPU times: user 34 µs, sys: 5 µs, total: 39 µs
Wall time: 49.8 µs


[]

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [12]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [15]:
print(corpus[:100])

[['v', 'i', 'l', 'l', 'e', 'de'], [], ['bruxelles'], [], ['bulletin', 'ires', '8eanas'], ['dl', '!'], [], ['conseil'], [], ['communal'], [], ['annee'], [], ['1847', '.'], [], ['au'], [], ['ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], [], ['bulletin'], [], ['conseil'], [], ['aes'], [], ['seances'], [], ['communal', '.'], [], ['annee'], [], ['1847', '.'], [], ['bruxelles', ','], ['imprimerie'], [], ['d', 'e', 'j', '.', 'h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ','], [], ['rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ','], [], ['1', '84', '8'], [], ['de', '!'], [], ['du', 'consei'], ['dibi'], [], ['e', '.', '-', 'communication'], ['conclusions', 'de', 'la', 'section', 'des'], ['du', 'nouvel_hospice', 'pour', 'les', 'av'], [], ['enraisonde', 'l', "'", 'absence', '&'], ['maladie', '.', 'le', 'conseil', 'ajourne'], ['leurs', 'de', 'pierre', 'el', 'marchai'], ['cles', 'des_taxes', 'communale', "'"], ['bieniaance', 'eldeseianv

## Entrainement d'un modèle Word2Vec sur ce corpus

In [16]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 13 s, sys: 517 ms, total: 13.5 s
Wall time: 6.84 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [17]:
outfile = f"../data/bulletins.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [18]:
model = Word2Vec.load("../data/bulletins.model")

### Imprimer le vecteur d'un terme

In [19]:
model.wv["bruxelles"]

array([-0.23398355, -0.559637  ,  2.6191323 ,  0.8283623 , -1.211586  ,
        2.7848547 , -0.20090796,  1.7191715 , -0.7548911 ,  0.17132838,
       -0.0898702 , -1.3116761 , -1.8711787 ,  1.1807737 ,  0.84744096,
       -0.15028816,  0.5488161 , -0.3780802 , -1.3439276 , -0.7205805 ,
       -0.4362293 ,  1.0497369 ,  1.55331   , -0.79254436,  1.4258304 ,
        0.25922304, -0.33397123, -0.06964107,  0.63124174, -0.6433839 ,
        0.40283933, -0.10081468], dtype=float32)

### Calculer la similarité entre deux termes

In [20]:
model.wv.similarity("commune", "bruxelles")

0.58437705

### Chercher les mots les plus proches d'un terme donné

In [21]:
model.wv.most_similar("boucherie", topn=10)

[('conservation', 0.9489635229110718),
 ('monnaie', 0.9452606439590454),
 ('gendarmerie', 0.9413977861404419),
 ('tour', 0.9403135776519775),
 ('chaussee', 0.9378595352172852),
 ('etablie', 0.9356231689453125),
 ('louvain', 0.9348666071891785),
 ('boulangerie_communale', 0.9342506527900696),
 ('navigation', 0.9341864585876465),
 ('philanthropie', 0.9340053796768188)]

### Faire des recherches complexes à travers l'espace vectoriel

In [22]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('hotel', 0.7868698835372925), ('paris', 0.7224560379981995), ('exercice_1847', 0.6903773546218872), ('interieur', 0.6849605441093445), ('hopital_st_.-_jean', 0.6799331307411194), ('etat', 0.6769092082977295), ('au_secretariat', 0.6642646193504333), ('enseignement_primaire', 0.660739004611969), ('gratuite', 0.6603370308876038), ('academie', 0.6596711874008179)]
